# Imports

In [27]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#export
from exp.nb_11 import *

# Serializing the model

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=2920)

In [3]:
path = untar_data(URLs.IMAGEWOOF_160)

In [4]:
size = 128
bs = 64

In [5]:
tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [6]:
len(il)

12954

In [7]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [8]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette, cuda=False)

In [9]:
create_phases??

Signature: create_phases(phases)
Docstring: <no docstring>
Source:   
def create_phases(phases):
    phases = listify(phases)
    return phases + [1-sum(phases)]
File:      ~/Documents/courses/fastai-courses/dl2/notebooks/exp/nb_11.py
Type:      function


In [10]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [11]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [12]:
learn.fit(1, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


/Users/imad/anaconda3/envs/fastbook/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

In [ ]:
st = learn.model.state_dict()

In [ ]:
type(st)

In [ ]:
', '.join(st.keys())

In [ ]:
st['10.bias']

In [19]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [ ]:
torch.save(st, mdl_path/'iw5')

# Pets

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3127)

In [ ]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [ ]:
pets.ls()

In [ ]:
pets_path = pets/'images'

In [ ]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [ ]:
il

In [ ]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [ ]:
random.seed(42)

In [ ]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [ ]:
sd

In [ ]:
n = il.items[0].name; n

In [ ]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

In [ ]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [ ]:
proc = CategoryProcessor()

In [ ]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [ ]:
', '.join(proc.vocab)

In [ ]:
ll.valid.x.tfms = val_tfms

In [ ]:
c_out = len(proc.vocab)

In [ ]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [ ]:
learn.fit(5, cbsched)

# Custom head

We always change the head of the model when doing transfer learning becuase the head was trained on specific task that is not related to our task.

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3265)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [ ]:
st = torch.load(mdl_path/'iw5')

In [ ]:
m = learn.model

In [ ]:
m.load_state_dict(st)

In [ ]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [ ]:
xb,yb = get_batch(data.valid_dl, learn)

In [ ]:
pred = m_cut(xb)

In [ ]:
pred.shape

In [ ]:
ni = pred.shape[1]

In [ ]:
#export
# Using both max and avg tends to boost performance a little bit and actually there is a paper about 
# this trick -> we concatenate both of them
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [15]:
torch.cat([torch.zeros(5, 10, 1, 1), torch.zeros(5, 10, 1, 1)], 1).shape

torch.Size([5, 20, 1, 1])

In [ ]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out)) # Linear layer needs 2x inputs becauce we concatenate max and avg pooling

In [ ]:
learn.model = m_new

In [ ]:
learn.fit(5, cbsched)

Transfrer learning gives huge boost out of the box.

# adapt_model and gradual unfreezing

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3483)

We first freeze the body and train the head and then unfreeze the body and trains all layers.

In [ ]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

In [ ]:
adapt_model(learn, data)

In [ ]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [ ]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

In [ ]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

It didn't work as well as the naive fine tuning that replaced the head and trained all layers together. The reason is because of __Batch Norm__. That is because at the beginning we freeze all layers including BN and the stats in those layers were using wolf images stats. Once we unfreeze, it starts to pick up the stats of the pet images.

The solution is to freeze all the body layers except the ones that have BN layers.

# Batch norm transfer

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3567)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [ ]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    # Unfreeze linear and BN at the beginning
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [ ]:
apply_mod(learn.model, partial(set_grad, b=False))

In [ ]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

In [ ]:
apply_mod(learn.model, partial(set_grad, b=True))

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

Pytorch already has an `apply` method we can use:

In [ ]:
learn.model.apply(partial(set_grad, b=False));

# Discriminative LR and param groups

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3799)

In [20]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [21]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/imad/.fastai/data/imagewoof2-160/models/iw5'

In [28]:
# This function basically creates two parameter groups:
# 1. BN and head of model
# 2. All other layers
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d):
            g2 += l.parameters()
        elif hasattr(l, "weight"):
            g1 += l.parameters()
        for ll in l.children():
            _bn_splitter(ll, g1, g2)

    g1, g2 = [], []
    _bn_splitter(m[0], g1, g2)

    g2 += m[1:].parameters()
    return g1, g2

In [ ]:
a,b = bn_splitter(learn.model)

In [ ]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [16]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [29]:
# export
from types import SimpleNamespace

cb_types = SimpleNamespace(**{o: o for o in Learner.ALL_CBS})

In [26]:
SimpleNamespace?

Init signature: SimpleNamespace(self, /, *args, **kwargs)
Docstring:     
A simple attribute-based namespace.

SimpleNamespace(**kwargs)
File:           ~/anaconda3/envs/fastbook/lib/python3.9/types.py
Type:           type
Subclasses:     


In [24]:
cb_types.after_backward

'after_backward'

In [25]:
cb_types

namespace(begin_fit='begin_fit',
          after_cancel_batch='after_cancel_batch',
          after_loss='after_loss',
          after_epoch='after_epoch',
          after_pred='after_pred',
          after_fit='after_fit',
          begin_epoch='begin_epoch',
          after_backward='after_backward',
          after_cancel_train='after_cancel_train',
          after_batch='after_batch',
          begin_batch='begin_batch',
          after_cancel_epoch='after_cancel_epoch',
          after_step='after_step',
          begin_validate='begin_validate')

In [30]:
# export
class DebugCallback(Callback):
    _order = 999

    def __init__(self, cb_name, f=None):
        self.cb_name, self.f = cb_name, f

    def __call__(self, cb_name):
        if cb_name == self.cb_name:
            if self.f:
                self.f(self.learner)
            else:
                set_trace()

In [31]:
# export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = [
        combine_scheds(phases, cos_1cycle_anneal(lr / 10.0, lr, lr / 1e5)) for lr in lrs
    ]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler("lr", sched_lr), ParamScheduler("mom", sched_mom)]

In [32]:
disc_lr_sched = sched_1cycle([0, 3e-2], 0.5)

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [ ]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

In [ ]:
learn.fit(3, disc_lr_sched)

In [ ]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [ ]:
learn.fit(5, disc_lr_sched)

# Export

In [34]:
!python ../src/notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exp/nb_11a.py
